In [1]:

from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API 
from tweepy import Cursor
from textblob import TextBlob
import re

import numpy as np
import pandas as pd
 
consumer_token = "RSV1mfohvaB3k3heHogWLJo7A"
consumer_secret = "ZPwMBhBtm5A1hd1O1KSwRKc9Pzd29Xz93H7Kzsz0xoGmXlFQ1X"
access_token = "938003970454306817-lV6LNuAYReogSr9wqy3yr9cUklphFtB"
access_secret = "iUgF09ZT1kzXEpESJagRfYXmeqkuXvazfWIBCLRaRGySG"

In [2]:


# # # # TWITTER CLIENT # # # #
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.twitter_user = twitter_user

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets
    
    def get_twitter_client_api(self):
        return self.twitter_client

    def get_friend_list(self, num_friends):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends, id=self.twitter_user).items(num_friends):
            friend_list.append(friend)
        return friend_list

    def get_home_timeline_tweets(self, num_tweets):
        home_timeline_tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline, id=self.twitter_user).items(num_tweets):
            home_timeline_tweets.append(tweet)
        return home_timeline_tweets


# # # # TWITTER AUTHENTICATER # # # #
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = OAuthHandler(consumer_token, consumer_secret)
        auth.set_access_token(access_token, access_secret)
        return auth

# # # # TWITTER STREAMER # # # #
class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self):
        self.twitter_autenticator = TwitterAuthenticator()    

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = TwitterListener(fetched_tweets_filename)
        auth = self.twitter_autenticator.authenticate_twitter_app() 
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords: 
        stream.filter(track=hash_tag_list)


# # # # TWITTER STREAM LISTENER # # # #
class TwitterListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          
    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)
        
class TweetAnalyzer():
    """
    Functionality for analyzing and categorizing content from tweets.
    """

    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

    def analyze_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        
        if analysis.sentiment.polarity > 0:
            return 1
        elif analysis.sentiment.polarity == 0:
            return 0.5
        else:
            return 0
#         return analysis.sentiment.polarity

    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['tweets'])

        df['id'] = np.array([tweet.id for tweet in tweets])
        df['len'] = np.array([len(tweet.text) for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['source'] = np.array([tweet.source for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df['retweets'] = np.array([tweet.retweet_count for tweet in tweets])

        return df

In [14]:
if __name__ == '__main__':

    twitter_client = TwitterClient()
    tweet_analyzer = TweetAnalyzer()

    api = twitter_client.get_twitter_client_api()

    tweets = api.user_timeline(screen_name="SatvikNema4", count=20)

    df = tweet_analyzer.tweets_to_data_frame(tweets)
    df['sentiment'] = np.array([tweet_analyzer.analyze_sentiment(tweet) for tweet in df['tweets']])

    print(df.head(10))

                                           tweets                   id  len  \
0                it was a very bad terrible movie  1192337605053927425   32   
1  Harry Potter is a very good movie. I liked it.  1192336563553755136   46   
2                            I do not like keshav  1185503328416829441   20   
3                           i liked the new movie  1185503258560716800   21   
4                                 hello like good  1185421084901826561   15   

                 date           source  likes  retweets  sentiment  
0 2019-11-07 07:06:52  Twitter Web App      0         0        0.0  
1 2019-11-07 07:02:43  Twitter Web App      0         0        1.0  
2 2019-10-19 10:29:53  Twitter Web App      0         0        0.5  
3 2019-10-19 10:29:37  Twitter Web App      0         0        1.0  
4 2019-10-19 05:03:05  Twitter Web App      0         0        1.0  


In [15]:
final = []
for i in tweets:
    temp = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", i.text).split()).lower() 
    final.append(temp)


In [16]:
final

['it was a very bad terrible movie',
 'harry potter is a very good movie i liked it',
 'i do not like keshav',
 'i liked the new movie',
 'hello like good']

In [17]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import time
from keras.backend import tensorflow_backend as K
from keras.datasets import imdb
model = tf.keras.models.load_model("model.h5")

In [18]:
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

In [19]:
final[0].split(" ")

['it', 'was', 'a', 'very', 'bad', 'terrible', 'movie']

In [20]:
toModel = []
x = []
for i in final:
    x=[]
    words = i.split(" ")
    for j in words:
        if j in word_index.keys():
            x.append(j)
            print(j," is present in the dictionary")
    toModel.append(x)
test = "i loved loved good like awesome"
test = test.lower().split(" ")
data = []

for tweet in toModel:
    x=[]
    for i in range(len(tweet)):
        print(tweet[i])
        t = word_index[tweet[i]]
        x.append(t)
    data.append(x)


it  is present in the dictionary
was  is present in the dictionary
a  is present in the dictionary
very  is present in the dictionary
bad  is present in the dictionary
terrible  is present in the dictionary
movie  is present in the dictionary
harry  is present in the dictionary
potter  is present in the dictionary
is  is present in the dictionary
a  is present in the dictionary
very  is present in the dictionary
good  is present in the dictionary
movie  is present in the dictionary
i  is present in the dictionary
liked  is present in the dictionary
it  is present in the dictionary
i  is present in the dictionary
do  is present in the dictionary
not  is present in the dictionary
like  is present in the dictionary
i  is present in the dictionary
liked  is present in the dictionary
the  is present in the dictionary
new  is present in the dictionary
movie  is present in the dictionary
hello  is present in the dictionary
like  is present in the dictionary
good  is present in the dictionary


In [21]:
word_index["rt"]

79189

In [22]:
data

[[12, 16, 6, 55, 78, 394, 20],
 [1335, 6507, 9, 6, 55, 52, 20, 13, 423, 12],
 [13, 81, 24, 40],
 [13, 423, 4, 162, 20],
 [4825, 40, 52]]

In [23]:
for i in range(len(data)):
    length = len(data[i]) 
    while length<256:
        data[i].append(0)
        length+=1
data = np.array(data)
for i in range(len(data)):
    print(len(data[i]))
    data[i] = np.array(data[i])
    data[i] = data[i].reshape(1,256)

256
256
256
256
256


In [24]:
type(data)

numpy.ndarray

In [25]:
print(type(data))
for i in data:
    print(type(i))
print()
print(data.shape)
for i in data:
    print(i.shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>

(5, 256)
(256,)
(256,)
(256,)
(256,)
(256,)


In [26]:
data.shape

(5, 256)

In [27]:
modelPrediction = model.predict(data)

In [13]:
final

['harry potter was a very good movie i liked it',
 'i do not like keshav',
 'i liked the new movie',
 'kartik is bad',
 'hello like good']

In [11]:
df[["tweets", "sentiment", "Model's output"]]

KeyError: '["Model\'s output"] not in index'

In [28]:
df["Model's output"] = modelPrediction

In [13]:
for i in modelPrediction:
    print(i)

[0.8713924]
[0.5010104]
[0.74521655]
[0.41664243]
[0.73302317]


In [29]:
df


,tweets,id,len,date,source,likes,retweets,sentiment,Model's output
0,it was a very bad terrible movie,1192337605053927425,32,2019-11-07 07:06:52,Twitter Web App,0,0,0.0,0.261429
1,Harry Potter is a very good movie. I liked it.,1192336563553755136,46,2019-11-07 07:02:43,Twitter Web App,0,0,1.0,0.884567
2,I do not like keshav,1185503328416829441,20,2019-10-19 10:29:53,Twitter Web App,0,0,0.5,0.501010
3,i liked the new movie,1185503258560716800,21,2019-10-19 10:29:37,Twitter Web App,0,0,1.0,0.745217
4,hello like good,1185421084901826561,15,2019-10-19 05:03:05,Twitter Web App,0,0,1.0,0.733023
